# Data Preprocessing

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openpyxl

In [62]:
# Importing survey data
df = pd.read_excel('data/Umfrage_online_18m.xlsx', index_col=0)
df

,Bitte wählen Sie Ihre bevorzugte Sprache aus / Please select your preferred language:,1. Geschlecht:,2. Alter:,3. Körpergröße (in cm),4. Körpergewicht (in kg),5. Wie bewerten Sie Ihr DERZEITIGES körperliches Befinden insgesamt?,6. Bitte bewerten Sie Ihr DERZEITIGES Temperaturempfinden auf den folgenden Skalen. Mir ist: [Gesamt],6. Bitte bewerten Sie Ihr DERZEITIGES Temperaturempfinden auf den folgenden Skalen. Mir ist: [Kopf],6. Bitte bewerten Sie Ihr DERZEITIGES Temperaturempfinden auf den folgenden Skalen. Mir ist: [Hände],6. Bitte bewerten Sie Ihr DERZEITIGES Temperaturempfinden auf den folgenden Skalen. Mir ist: [Füße],...,7. Ich empfinde die Temperatur- und Luftverhältnisse in meiner Umgebung als: [Hände],7. Ich empfinde die Temperatur- und Luftverhältnisse in meiner Umgebung als: [Füße],8. Wie lange fahren Sie schon auf dem Bus?,Kopf und Hände,Öberkörper,Beine,Füße,10. Bitte geben Sie an ob Sie sitzen oder stehen:,11. Bitte geben Sie Ihre Position im Bus an:,"12. Bitte geben Sie an, ob sie näher einer Fenster oder dem Gang sind:"
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-11-23 08:53:51,Deutsch / German,Weiblich,62,162,60,Mittel,Leicht kühl,Leicht kühl,Leicht kühl,Leicht kühl,...,Leicht unangenehm,Leicht unangenehm,Weniger als 15 Minuten,Schal,"Pullover, dünn;Jacke, dick","Hose, lang","Socke, dick;Winterschuhe",Ich sitze,Vorderwagen - Vordere Tür,Gang
2021-11-23 08:57:47,Deutsch / German,Weiblich,36,163,65,Mittel,Neutral,Neutral,Neutral,Neutral,...,Angenehm,Angenehm,Weniger als 15 Minuten,Mütze,"Pullover, dick","Hose, lang","Socke, dünn;Stiefel",Ich sitze,Hinterwagen - Hinterbereich,Gang
2021-11-23 09:12:25,Deutsch / German,Weiblich,23,168,68,Schlecht,Leicht kühl,Leicht kühl,Kühl,Kalt,...,Angenehm,Angenehm,15 bis 30 Minuten,Keine der oben gezeigten Optionen,"T-Shirt;Pullover, dick;Jacke, dünn","Hose, lang;Strumpfhose","Socke, dünn;Sommerschuhe",Ich sitze,Vorderwagen - Zwischen den Türen,Gang
2021-11-23 09:15:20,Deutsch / German,Weiblich,22,178,80,Mittel,Neutral,Neutral,Neutral,Neutral,...,Leicht unangenehm,Leicht unangenehm,Weniger als 15 Minuten,Schal;Mütze,"Pullover, dünn;Jacke, dünn","Hose, lang","Socke, dünn;Sommerschuhe",Ich sitze,Vorderwagen - Hintere Tür,Fenster
2021-11-23 09:17:59,Deutsch / German,Weiblich,25,163,65,Schlecht,Neutral,Neutral,Leicht kühl,Kühl,...,Angenehm,Angenehm,Weniger als 15 Minuten,Schal,"Pullover, dünn;Jacke, dick","Hose, lang","Socke, dünn;Sommerschuhe",Ich sitze,Hinterwagen - Tür,Fenster
2021-11-23 09:19:28,Deutsch / German,Weiblich,27,168,57,Mittel,Neutral,Neutral,Kühl,Kühl,...,Angenehm,Angenehm,30 bis 45 Minuten,Keine der oben gezeigten Optionen,"Pullover, dick;Jacke, dünn","Hose, lang","Socke, dünn;Sommerschuhe",Ich sitze,Hinterwagen - Tür,Fenster
2021-11-23 09:32:25,Deutsch / German,Männlich,38,187,69,Mittel,Neutral,Neutral,Neutral,Neutral,...,Angenehm,Angenehm,Weniger als 15 Minuten,Keine der oben gezeigten Optionen,"Pullover, dünn;Jacke, dünn","Hose, lang","Socke, dünn;Sommerschuhe",Ich sitze,Gelenkbereich,Gang
2021-11-23 09:36:41,Deutsch / German,Männlich,73,167,83,Gut,Neutral,Neutral,Neutral,Neutral,...,Angenehm,Angenehm,Weniger als 15 Minuten,Mütze,"T-Shirt;Jacke, dick","Hose, lang","Socke, dick;Winterschuhe",Ich sitze,Gelenkbereich,Fenster
2021-11-23 10:02:34,Deutsch / German,Männlich,61,176,80,Mittel,Leicht kühl,Kühl,Neutral,Neutral,...,Angenehm,Angenehm,Weniger als 15 Minuten,Keine der oben gezeigten Optionen,"Pullover, dick;Jacke, dünn","Hose, lang","Socke, dünn;Winterschuhe",Ich sitze,Hinterwagen - Tür,Gang


In [63]:
interesting_columns = [
 '1. Geschlecht:',
 '2. Alter:',
 '3. Körpergröße (in cm)',
 '4. Körpergewicht (in kg)',
 '5. Wie bewerten Sie Ihr DERZEITIGES körperliches Befinden insgesamt?',
 '6. Bitte bewerten Sie Ihr DERZEITIGES Temperaturempfinden auf den folgenden Skalen. Mir ist: [Gesamt]',
 '7. Ich empfinde die Temperatur- und Luftverhältnisse in meiner Umgebung als: [Gesamt]',
 '8. Wie lange fahren Sie schon auf dem Bus?',
 'Kopf und Hände',
 'Öberkörper',
 'Beine',
 'Füße',
 '10. Bitte geben Sie an ob Sie sitzen oder stehen:',
 '11. Bitte geben Sie Ihre Position im Bus an:',
 '12. Bitte geben Sie an, ob sie näher einer Fenster oder dem Gang sind:']

In [64]:
df = df[interesting_columns]

In [65]:
# Defining categorical data to convert in numerical values
categorical_columns = [
 '1. Geschlecht:',
 '5. Wie bewerten Sie Ihr DERZEITIGES körperliches Befinden insgesamt?',
 '6. Bitte bewerten Sie Ihr DERZEITIGES Temperaturempfinden auf den folgenden Skalen. Mir ist: [Gesamt]',
 '7. Ich empfinde die Temperatur- und Luftverhältnisse in meiner Umgebung als: [Gesamt]',
 '8. Wie lange fahren Sie schon auf dem Bus?',
 '10. Bitte geben Sie an ob Sie sitzen oder stehen:',
 '11. Bitte geben Sie Ihre Position im Bus an:',
 '12. Bitte geben Sie an, ob sie näher einer Fenster oder dem Gang sind:']

In [66]:
categorical_dict = {}
for column in categorical_columns:
 map_dict = {}
 for i, value in enumerate(df[column].unique()):
  map_dict[value] = i
 categorical_dict[column] = map_dict

In [67]:
categorical_dict['5. Wie bewerten Sie Ihr DERZEITIGES körperliches Befinden insgesamt?'] = \
 {'Mittel': 1,
  'Schlecht': 0,
  'Gut': 2,
  'Sehr gut': 3}
categorical_dict['6. Bitte bewerten Sie Ihr DERZEITIGES Temperaturempfinden auf den folgenden Skalen. Mir ist: [Gesamt]'] =  {'Leicht kühl': 2,
  'Neutral': 3,
  'Kühl': 1,
  'Kalt': 0,
  'Leicht warm': 4,
  'Warm': 5}

In [69]:
df = df.replace(categorical_dict)

In [70]:
# Using clothing_columns to define user insulation value
clothing_columns = [
 'Kopf und Hände',
 'Öberkörper',
 'Beine',
 'Füße']

In [71]:
# Separate string column
df[clothing_columns] = df[clothing_columns].apply(lambda x: x.str.split(';'))

In [72]:
# Join columns
df['clothing'] = df['Kopf und Hände'] + df['Öberkörper'] + df['Beine'] + df['Füße']
# Drop columns
df.drop(clothing_columns, axis=1, inplace=True)

In [73]:
clothing_items = []
for index, values in df['clothing'].iteritems():
 clothing_items += values
 clothing_items = list(set(clothing_items))

In [74]:
def clothing_thermal_insulation(x):
 """
 Calculate thermal insulation value from cloth used

 :param x: list of clothing items
 :return: insulation value
 """
 insulation = 0
 insulation_dict = {'Strumpfhose': 3,
 'Sommerschuhe': 2,
 'Übergangsschuhe': 3,
 'Hemd / Bluse, langer Arm': 6,
 'Jacke, dünn': 8,
 'Mütze': 2,
 'Handschuhe': 2,
 'Schal': 2,
 'Stiefel': 3,
 'Jogginghose': 4,
 'Socke, dick': 3,
 'Mantel': 9,
 'Pullover, dick': 9,
 'Hose, lang': 6,
 'Pullover, dünn': 5,
 'Socke, dünn': 1,
 'Keine der oben gezeigten Optionen': 0,
 'Winterschuhe': 5,
 'Jacke, dick': 10,
 'T-Shirt': 6}
 for item in x:
  insulation += insulation_dict[item]
 return insulation
df['clothing_insulation'] = df['clothing'].apply(clothing_thermal_insulation)

In [75]:
# Sumarazing comfort related questions to one mean value
df['comfort'] = df[['5. Wie bewerten Sie Ihr DERZEITIGES körperliches Befinden insgesamt?',
 '6. Bitte bewerten Sie Ihr DERZEITIGES Temperaturempfinden auf den folgenden Skalen. Mir ist: [Gesamt]',
 '7. Ich empfinde die Temperatur- und Luftverhältnisse in meiner Umgebung als: [Gesamt]']].mean(axis=1)
df.drop(['5. Wie bewerten Sie Ihr DERZEITIGES körperliches Befinden insgesamt?',
 '6. Bitte bewerten Sie Ihr DERZEITIGES Temperaturempfinden auf den folgenden Skalen. Mir ist: [Gesamt]',
 '7. Ich empfinde die Temperatur- und Luftverhältnisse in meiner Umgebung als: [Gesamt]', 'clothing'], axis=1, inplace=True)


In [86]:
df

,1. Geschlecht:,2. Alter:,3. Körpergröße (in cm),4. Körpergewicht (in kg),8. Wie lange fahren Sie schon auf dem Bus?,10. Bitte geben Sie an ob Sie sitzen oder stehen:,11. Bitte geben Sie Ihre Position im Bus an:,"12. Bitte geben Sie an, ob sie näher einer Fenster oder dem Gang sind:",clothing_insulation,comfort
Timestamp,,,,,,,,,,
2021-11-23 08:53:51,0,62,162,60,0,0,0.0,0,31,1.000000
2021-11-23 08:57:47,0,36,163,65,0,0,1.0,0,21,1.666667
2021-11-23 09:12:25,0,23,168,68,1,0,2.0,0,35,1.000000
2021-11-23 09:15:20,0,22,178,80,0,0,3.0,1,26,1.333333
2021-11-23 09:17:59,0,25,163,65,0,0,4.0,1,26,1.333333
2021-11-23 09:19:28,0,27,168,57,2,0,4.0,1,26,1.666667
2021-11-23 09:32:25,1,38,187,69,0,0,5.0,0,22,1.666667
2021-11-23 09:36:41,1,73,167,83,0,0,5.0,1,32,2.000000
2021-11-23 10:02:34,1,61,176,80,0,0,4.0,0,29,1.333333


## Linear Regression

In [76]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.optim as optim

In [77]:
### Load the data
input_data = torch.tensor(df.drop(['comfort'], axis=1).values, dtype=torch.float)
output_data = torch.tensor(df['comfort'].values, dtype=torch.float)


In [78]:
output_data = output_data.reshape(-1, 1)

In [79]:
# And make a convenient variable to remember the number of input columns
n = input_data.shape[1]


### Model definition

# First we define the trainable parameters A and b
A = torch.randn((n, 1), requires_grad=True)
b = torch.randn(1, requires_grad=True)

# Then we define the prediction model
def model(x_input):
    return torch.mm(x_input,A) + b


### Loss function definition ###

def loss(y_predicted, y_target):
    return ((y_predicted - y_target)**2).sum()

### Training the model ###

# Setup the optimizer object, so it optimizes a and b.
optimizer = optim.Adam([A, b], lr=0.1)

# Main optimization loop
for t in range(2000):
    # Set the gradients to 0.
    optimizer.zero_grad()
    # Compute the current predicted y's from x_dataset
    predicted = model(input_data)
    # See how far off the prediction is
    current_loss = loss(predicted, output_data)
    # Compute the gradient of the loss with respect to A and b.
    current_loss.backward()
    # Update A and b accordingly.
    optimizer.step()
    print(f"t = {t}, loss = {current_loss}, A = {A.detach().numpy()}, b = {b.item()}")

t = 0, loss = 24455.11328125, A = [[-1.2443522 ]
 [-0.96813715]
 [ 0.68729275]
 [-0.17721644]
 [ 1.467245  ]
 [-0.48229584]
 [-1.0728098 ]
 [-0.19877371]
 [-1.2873269 ]], b = 0.9347772598266602
t = 1, loss = 41269.08984375, A = [[-1.3130678 ]
 [-0.9558582 ]
 [ 0.63453645]
 [-0.23060733]
 [ 1.4296259 ]
 [-0.52703863]
 [-1.1098881 ]
 [-0.25555992]
 [-1.3335555 ]], b = 0.8841701149940491
t = 2, loss = 22619.5078125, A = [[-1.3895435 ]
 [-0.94582117]
 [ 0.5721974 ]
 [-0.29349157]
 [ 1.3822699 ]
 [-0.5815569 ]
 [-1.1566532 ]
 [-0.32161704]
 [-1.389576  ]], b = 0.8238741755485535
t = 3, loss = 19369.71484375, A = [[-1.4489576 ]
 [-0.9055374 ]
 [ 0.5341481 ]
 [-0.33218896]
 [ 1.3647621 ]
 [-0.60838795]
 [-1.1733458 ]
 [-0.365078  ]
 [-1.4185575 ]], b = 0.7887901663780212
t = 4, loss = 24555.60546875, A = [[-1.4799082 ]
 [-0.8441705 ]
 [ 0.53009313]
 [-0.33685765]
 [ 1.3818617 ]
 [-0.6002558 ]
 [-1.1554435 ]
 [-0.37539247]
 [-1.4126921 ]], b = 0.7880656123161316
t = 5, loss = 20821.86328125, A

In [80]:
A

tensor([[ 0.3213],
        [-0.0049],
        [ 0.0070],
        [ 0.0038],
        [-0.0240],
        [-0.1522],
        [-0.1092],
        [ 0.5370],
        [-0.0257]], requires_grad=True)

In [81]:
df.columns

Index(['1. Geschlecht:', '2. Alter:', '3. Körpergröße (in cm)',
       '4. Körpergewicht (in kg)',
       '8. Wie lange fahren Sie schon auf dem Bus?',
       '10. Bitte geben Sie an ob Sie sitzen oder stehen:',
       '11. Bitte geben Sie Ihre Position im Bus an:',
       '12. Bitte geben Sie an, ob sie näher einer Fenster oder dem Gang sind:',
       'clothing_insulation', 'comfort'],
      dtype='object')

## Neural Network

In [82]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn

class Network(nn.Module):
    def __init__(self):
        super().__init__()

        # Inputs to hidden layer linear transformation
        self.hidden = nn.Linear(9, 5)
        # Output layer, 10 units - one for each digit
        self.output = nn.Linear(5, 1)

        # Define sigmoid activation and softmax output
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Pass the input tensor through each of our operations
        x = self.hidden(x)
        x = self.sigmoid(x)
        x = self.output(x)
        x = self.sigmoid(x)
        return x

In [83]:
df_normalized = (df-df.min())/(df.max()-df.min())

In [84]:
### Load the data
input_data = torch.tensor(df_normalized.drop(['comfort'], axis=1).values, dtype=torch.float)
target_data = torch.tensor(df_normalized['comfort'].values, dtype=torch.float).view(-1, 1)


In [85]:
import torch.optim as optim

net = Network()

criterion = nn.MSELoss()
# create your optimizer
optimizer = optim.Adam(net.parameters(), lr=0.01)

# in your training loop:
epochs = 1000
for epoch in range(epochs):
    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    # Forward pass: Compute predicted y by passing x to the model
    output = net(input_data)

    # Compute and print loss
    loss = criterion(output, target_data)
    print(f"Epoch {epoch+1}/{epochs} loss: {loss.item()}")
    loss.backward()
    optimizer.step()

Epoch 1/1000 loss: 0.06954556703567505
Epoch 2/1000 loss: 0.06690232455730438
Epoch 3/1000 loss: 0.0644325539469719
Epoch 4/1000 loss: 0.06214753910899162
Epoch 5/1000 loss: 0.06005573272705078
Epoch 6/1000 loss: 0.05816170200705528
Epoch 7/1000 loss: 0.05646578595042229
Epoch 8/1000 loss: 0.05496535450220108
Epoch 9/1000 loss: 0.05365617945790291
Epoch 10/1000 loss: 0.05253225937485695
Epoch 11/1000 loss: 0.05158524215221405
Epoch 12/1000 loss: 0.05080408975481987
Epoch 13/1000 loss: 0.050175298005342484
Epoch 14/1000 loss: 0.04968322813510895
Epoch 15/1000 loss: 0.049310702830553055
Epoch 16/1000 loss: 0.049039460718631744
Epoch 17/1000 loss: 0.048850711435079575
Epoch 18/1000 loss: 0.04872574657201767
Epoch 19/1000 loss: 0.048646535724401474
Epoch 20/1000 loss: 0.04859631136059761
Epoch 21/1000 loss: 0.04856012016534805
Epoch 22/1000 loss: 0.04852525517344475
Epoch 23/1000 loss: 0.048481520265340805
Epoch 24/1000 loss: 0.04842137545347214
Epoch 25/1000 loss: 0.048339903354644775
Epo

# Todo:
- analysis de corelacion
- 2 nodes model pg 200 (79)
- model de fanger, pg 198
- Thermische_Behaglichkeit_global_single_Position
- Meas- sensores "estaciones"
- ISO 7730
- equivalent temperature
-